In [1]:

## import certain packages
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date, timedelta
import numpy as np
import pandas as pd
from pandas import DataFrame
from wallstreet import Stock, Call, Put

In [2]:
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/sigma*sqrt(T)
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)


In [3]:
## define the call options price function
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

In [4]:
## define the put options price function
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

In [5]:
## define the Call_Greeks of an option
def call_delta(S,K,T,r,sigma):
    return norm.cdf(d1(S,K,T,r,sigma))
def call_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))

In [6]:
## define the Put_Greeks of an option
def put_delta(S,K,T,r,sigma):
    return -norm.cdf(-d1(S,K,T,r,sigma))
def put_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))
def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))

In [7]:
SPY_TICKER = "SPY"
spy = Stock(SPY_TICKER)
p = Put(SPY_TICKER)

No options listed for given date, using 07-05-2021 instead


In [8]:
nextMonth = date.today() + timedelta(days=+31)
expirations = list(map(lambda x: datetime.strptime(x, "%d-%m-%Y"), p.expirations))
exp = min(list(filter(lambda x: (x.date() - nextMonth).days >= 0, expirations)))
strike = min(list(filter(lambda x: x >= spy.price, p.strikes)))


In [9]:
spyc = Call(SPY_TICKER, d=exp.day, m=exp.month, y=exp.year, strike=strike)
spyp = Put(SPY_TICKER, d=exp.day, m=exp.month, y=exp.year, strike=strike)
spyp

Put(ticker=SPY, expiration=07-06-2021, strike=416)

In [10]:
S = spy.price
K = strike
T = (exp - datetime.utcnow()).days / 365
r = 20 #???
sigma = 20 #???

In [11]:
data = {'Symbol': ['S', 'K', 'T', 'r', 'sigma'],
        'Input': [S, K, T , r , sigma]}
input_frame = DataFrame(data, columns=['Symbol', 'Input'], 
                   index=['Underlying price', 'Strike price', 'Time to maturity', 'Risk-free interest rate', 'Volatility'])
input_frame

,Symbol,Input
Underlying price,S,415.750000
Strike price,K,416.000000
Time to maturity,T,0.084932
Risk-free interest rate,r,20.000000
Volatility,sigma,20.000000


In [12]:
r = r/100; sigma = sigma/100;
price_and_greeks = {'Call' : [bs_call(S,K,T,r,sigma), 
                              call_delta(S,K,T,r,sigma), 
                              call_gamma(S,K,T,r,sigma),
                              call_vega(S,K,T,r,sigma), 
                              call_rho(S,K,T,r,sigma), 
                              call_theta(S,K,T,r,sigma)],
                    'Put' : [bs_put(S,K,T,r,sigma), 
                             put_delta(S,K,T,r,sigma), 
                             put_gamma(S,K,T,r,sigma),
                             put_vega(S,K,T,r,sigma), 
                             put_rho(S,K,T,r,sigma), 
                             put_theta(S,K,T,r,sigma)]}
price_and_greeks_frame = DataFrame(price_and_greeks, columns=['Call','Put'], index=['Price', 'delta', 'gamma','vega','rho','theta'])
price_and_greeks_frame

TypeError: 'datetime.datetime' object is not callable